In [ ]:
from PIL import Image
import torch.nn as nn
import os
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image
#All images follow this format Abstract_image_155
#up to image 2781

#batch size
bs = 32

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)


In [1]:

#not used
img_size = 200

print('test')
def noise(size):
    return Variable(torch.randn(size, 1*28*28))

class Generator(nn.Module):
    def __init__(self, z_dim=28*28, out_dim=28*28):
        super().__init__()
        print('Creating Generator ')
        self.gen = nn.Sequential(
            nn.Linear(z_dim, (28*28)*2),
            nn.LeakyReLU(0.2),
            #We want a 28*28 output
            nn.Linear((28*28)*2, 28*28),
            #We want an output of -1 to 1, Tanh() will work for this
            nn.Tanh()
        )

    def forward(self, x):
        return self.gen(x)

class Discriminator(nn.Module):
    #z_dim must match the output of the generator, in this case 28*28
    def __init__(self, z_dim=28*28):
        super().__init__()
        self.dis = nn.Sequential(
            nn.Linear(z_dim, 28*28*2),
            nn.LeakyReLU(0.2),
            nn.Linear(28*28*2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.dis(x)  

class GAN:
    def __init__(self, discriminator, generator, batch_size=32):
        self.generator = generator
        self.discriminator = discriminator
        self.batch_size = batch_size

        # Define binary cross entropy loss
        self.loss = nn.BCELoss()

        # Define separate optimizers for discriminator and generator
        self.d_optimizer = torch.optim.Adam(self.discriminator.parameters(), lr=0.0002)
        self.g_optimizer = torch.optim.Adam(self.generator.parameters(), lr=0.0002)

    def train(self, num_epochs, dataloader):
        # Training Loop for each epoch
        print ('Starting')
        for epoch in range(num_epochs):
            
            if epoch % 10 == 0:
                # Generate images from noise, using the generator network.
                sample_vectors = noise(64)
                samples = generator(sample_vectors)
                print(samples.shape)  # add print statement to check the shape of samples
                samples = samples.view(64, 1, 28, 28) # reshape to (batch_size, channels, height, width)
                print(samples.shape)  # add print statement to check the shape of samples after reshaping
                
                #CHANGE TO THE DIRECTORY YOU WANT
                save_image(samples, f'martmine/output_{epoch}.png', normalize=True)
            # Batch Loop for each set of images and labels
            for n, (images, _) in enumerate(dataloader):
                
                # Prepare real images and real labels
                real_images = Variable(images.view(images.size(0), -1))
                real_labels = Variable(torch.ones(self.batch_size, 1))

                # Train Discriminator on real images
                self.d_optimizer.zero_grad()
                real_outputs = self.discriminator(real_images)
                d_loss_real = self.loss(real_outputs, real_labels)
                d_loss_real.backward()
                
                # Prepare fake images and fake labels
                noise_vectors = noise(self.batch_size)
                fake_images = self.generator(noise_vectors)
                fake_labels = Variable(torch.zeros(self.batch_size, 1))

                # Train Discriminator on fake images
                fake_outputs = self.discriminator(fake_images)
                d_loss_fake = self.loss(fake_outputs, fake_labels)
                d_loss_fake.backward()

                # Update Discriminator weights
                self.d_optimizer.step()

                # Train Generator to fool the Discriminator
                self.g_optimizer.zero_grad()
                noise_vectors = noise(self.batch_size)
                fake_images = self.generator(noise_vectors)
                outputs = self.discriminator(fake_images)
                
                # We train the generator to generate images that the discriminator will think are real
                g_loss = self.loss(outputs, Variable(torch.ones(self.batch_size, 1)))
                g_loss.backward()

                # Update Generator weights
                self.g_optimizer.step()

                if (n+1) % 1 == 0:
                    print(f'Epoch {epoch+1}/{num_epochs}, Step {n+1}, d_loss: {d_loss_real+d_loss_fake}, g_loss: {g_loss}')
    
        
        

generator = Generator()
discriminator = Discriminator()

gan = GAN(discriminator, generator)
gan.train(2000, train_loader)

test


NameError: name 'nn' is not defined

### 